In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import backend as K
from collections import defaultdict

In [2]:
url = "https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip"
path_to_zip = tf.keras.utils.get_file("images_background.zip", origin=url, extract=True)
path_to_zip

'C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted'

In [3]:
IMG_SIZE = 105  # Image size for resizing
BATCH_SIZE = 32
EPOCHS = 20

In [4]:
def load_image(image_path):
    if not os.path.isfile(image_path):
        print(f"Skipped: {image_path} is a directory.")
        return None  # Skip directories
    
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # Normalize to [0, 1]
    return image

def get_image_paths_and_labels(base_dir):
    image_paths = []
    labels = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.png') or file.endswith('.jpg'):
                image_paths.append(os.path.join(root, file))
                labels.append(root.split('\\')[-2])  # Character as label
    return image_paths, labels

# Update the path to your dataset directory
base_dir = os.path.join(path_to_zip,'images_background')
base_dir = os.path.join(base_dir, 'train')
image_paths, labels = get_image_paths_and_labels(base_dir)
print(f"Total images: {len(image_paths)}, Total labels: {len(set(labels))}")


Total images: 17600, Total labels: 27


In [5]:
from sklearn.preprocessing import LabelEncoder

def prepare_data(image_paths, labels):
    images = []
    valid_paths = []
    for path in image_paths:
        img = load_image(path)
        if img is not None:
            images.append(img)
            valid_paths.append(path)
    return np.array(images), valid_paths

X, valid_paths = prepare_data(image_paths, labels)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform([path.split('\\')[-3] for path in valid_paths])
print(y)

[ 0  0  0 ... 26 26 26]


In [6]:
def create_pairs(images, labels):
    pairs = []
    pair_labels = []
    num_classes = len(np.unique(labels))
    
    # Create pairs
    for idx in range(len(images)):
        current_img = images[idx]
        current_label = labels[idx]
        
        # Positive pair
        pos_idx = np.random.choice(np.where(labels == current_label)[0])
        pos_img = images[pos_idx]
        pairs.append([current_img, pos_img])
        pair_labels.append(1)
        
        # Negative pair
        neg_label = (current_label + np.random.randint(1, num_classes)) % num_classes
        neg_idx = np.random.choice(np.where(labels == neg_label)[0])
        neg_img = images[neg_idx]
        pairs.append([current_img, neg_img])
        pair_labels.append(0)
    
    return np.array(pairs), np.array(pair_labels)

pairs, pair_labels = create_pairs(X, y)


In [7]:
def build_feature_extractor():
    base_model = ResNet50(weights=None, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    base_model.trainable = False
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    output = layers.Dense(256, activation='relu')(x)
    #output = layers.Dense(len(set(labels)), activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = build_feature_extractor()
#model.summary()


In [8]:
class EuclideanDistance(layers.Layer):
    def __init__(self, **kwargs):
        super(EuclideanDistance, self).__init__(**kwargs)

    def call(self, inputs):
        x1, x2 = inputs
        return K.sqrt(K.sum(K.square(x1 - x2), axis=-1, keepdims=True))

In [9]:
def contrastive_loss(y_true, y_pred, margin=0.10):
    squared_pred = K.square(y_pred)  # D^2
    margin_square = K.square(K.maximum(margin - y_pred, 0))  # (margin - D)^2
    return K.mean((1 - y_true) * squared_pred + y_true * margin_square)

$L2 =\sqrt{\sum \limits _{j=1} ^{n}({x_1}_i - {x_2}_i)^2}$

In [10]:
def build_siamese_network():
    feature_extractor = build_feature_extractor()
    
    input_a = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    input_b = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    
    feat_a = feature_extractor(input_a)
    feat_b = feature_extractor(input_b)
    
    # Compute L2 distance
    l2_distance = layers.Lambda(lambda tensors: K.sqrt(K.sum(K.square(tensors[0] - tensors[1]), axis=-1, keepdims=True)))([feat_a, feat_b])
    
    # Siamese Network
    siamese_network = models.Model(inputs=[input_a, input_b], outputs=l2_distance)
    
    # Compile with Contrastive Loss
    siamese_network.compile(optimizer='adam', loss=contrastive_loss, metrics=['accuracy'])
    
    return siamese_network


In [11]:
X_a = pairs[:, 0]
X_b = pairs[:, 1]

X_a = np.stack(X_a, axis=0)
X_b = np.stack(X_b, axis=0)

X_train_a, X_val_a, X_train_b, X_val_b, y_train, y_val = train_test_split(X_a, X_b, pair_labels, test_size=0.2, random_state=42)


In [12]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="best_model_refactor.h5",   
    save_best_only=True, 
    monitor="val_loss",
    mode="min",
    verbose=1,
)

In [13]:
siamese_network = build_siamese_network()

history = siamese_network.fit(
    [X_train_a, X_train_b], y_train,
    validation_data=([X_val_a, X_val_b], y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)



Epoch 1/20
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step - accuracy: 0.4983 - loss: 0.0301
Epoch 1: val_loss improved from inf to 0.00503, saving model to best_model_refactor.h5


880/880 ━━━━━━━━━━━━━━━━━━━━ 396s 432ms/step - accuracy: 0.4983 - loss: 0.0301 - val_accuracy: 0.4972 - val_loss: 0.0050
Epoch 2/20
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step - accuracy: 0.5021 - loss: 0.0050
Epoch 2: val_loss did not improve from 0.00503
880/880 ━━━━━━━━━━━━━━━━━━━━ 381s 433ms/step - accuracy: 0.5021 - loss: 0.0050 - val_accuracy: 0.4972 - val_loss: 0.0050
Epoch 3/20
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - accuracy: 0.5025 - loss: 0.0050
Epoch 3: val_loss did not improve from 0.00503
880/880 ━━━━━━━━━━━━━━━━━━━━ 374s 425ms/step - accuracy: 0.5025 - loss: 0.0050 - val_accuracy: 0.4972 - val_loss: 0.0050
Epoch 4/20
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - accuracy: 0.5022 - loss: 0.0050
Epoch 4: val_loss did not improve from 0.00503
880/880 ━━━━━━━━━━━━━━━━━━━━ 374s 425ms/step - accuracy: 0.5022 - loss: 0.0050 - val_accuracy: 0.4972 - val_loss: 0.0050
Epoch 5/20
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 0.4974 - loss: 0.0050
Epoch 5: val_loss did n

KeyboardInterrupt: 

In [14]:
loss, accuracy = siamese_network.evaluate([X_val_a, X_val_b], y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")


220/220 ━━━━━━━━━━━━━━━━━━━━ 74s 336ms/step - accuracy: 0.4981 - loss: 0.5019
Validation Loss: 0.5028409361839294, Validation Accuracy: 0.49715909361839294


In [ ]:
siamese_network.save("model_weights.h5")

In [15]:
siamese_network.load_weights('best_model_refactor.h5')

In [20]:
######### inference ########


import tensorflow as tf


NUM_WAYS = 3     # 3 classes per episode
NUM_SHOTS = 2    # 2 examples per class for support set
NUM_QUERIES = 3  # 1 query example per class (you'll use your 3 new images)


# Load the trained model
siamese_net = siamese_network

base_dir = os.path.join(path_to_zip,'images_background')
base_dir = os.path.join(base_dir, 'val')

# Manually selected paths for the support set (2 images per class, 3 classes)
support_image_paths = [
    # Class 0 images / Balinese
    os.path.join(base_dir, 'Balinese\\character01\\0108_01.png'),
    os.path.join(base_dir, 'Balinese\\character01\\0108_02.png'),

    # Class 1 images /  Bengali
    os.path.join(base_dir, 'Bengali\\character08\\0139_01.png'),
    os.path.join(base_dir, 'Bengali\\character08\\0139_12.png'),

    # Class 2 images / Blackfoot
    os.path.join(base_dir, 'Blackfoot_(Canadian_Aboriginal_Syllabics)\\character04\\0181_04.png'),
    os.path.join(base_dir, 'Blackfoot_(Canadian_Aboriginal_Syllabics)\\character06\\0183_06.png'),
]

# Corresponding labels for the support set
support_labels = [
    0, 0,  # Class 0
    1, 1,  # Class 1
    2, 2   # Class 2
]

support_image_paths

['C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted\\images_background\\val\\Balinese\\character01\\0108_01.png',
 'C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted\\images_background\\val\\Balinese\\character01\\0108_02.png',
 'C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted\\images_background\\val\\Bengali\\character08\\0139_01.png',
 'C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted\\images_background\\val\\Bengali\\character08\\0139_12.png',
 'C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted\\images_background\\val\\Blackfoot_(Canadian_Aboriginal_Syllabics)\\character04\\0181_04.png',
 'C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted\\images_background\\val\\Blackfoot_(Canadian_Aboriginal_Syllabics)\\character06\\0183_06.png']

In [21]:
def load_and_preprocess(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # Normalize to [0, 1]
    return image

def prepare_manual_support_set(support_image_paths, support_labels):
    support_images = [load_and_preprocess(img) for img in support_image_paths]
    support_images = np.stack(support_images)
    support_labels = np.array(support_labels)
    return support_images, support_labels

support_images, support_labels = prepare_manual_support_set(support_image_paths, support_labels)
print(f"Support Images Shape: {support_images.shape}, Labels Shape: {support_labels.shape}")


Support Images Shape: (6, 105, 105, 3), Labels Shape: (6,)


In [22]:
# Manually selected paths for the query set (3 new images to classify)

query_image_paths = [
    
    # Class 2 images / Blackfoot
    os.path.join(base_dir, 'Blackfoot_(Canadian_Aboriginal_Syllabics)\\character04\\0181_05.png'),

   # Class 1 images / Bengali
   os.path.join(base_dir, 'Bengali\\character08\\0139_03.png'),

   
   # Class 0 images / Balinese
   os.path.join(base_dir, 'Balinese\\character01\\0108_08.png'),
]

def prepare_query_images(query_image_paths):
    print(query_image_paths)
    query_images = [load_and_preprocess(img) for img in query_image_paths]
    return np.stack(query_images)

query_images = prepare_query_images(query_image_paths)
print(f"Query Images Shape: {query_images.shape}")


['C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted\\images_background\\val\\Blackfoot_(Canadian_Aboriginal_Syllabics)\\character04\\0181_05.png', 'C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted\\images_background\\val\\Bengali\\character08\\0139_03.png', 'C:\\Users\\lariyan.pouya\\.keras\\datasets\\images_background_extracted\\images_background\\val\\Balinese\\character01\\0108_08.png']
Query Images Shape: (3, 105, 105, 3)


In [25]:
support_labels

array([0, 0, 1, 1, 2, 2])

In [ ]:
def classify_images(support_images, support_labels, query_images, model):
    predictions = []
    
    for query_img in query_images:
        query_img = tf.expand_dims(query_img, axis=0)  # Expand dims for batch compatibility

        tiled_query = np.tile(query_img, (support_images.shape[0], 1, 1, 1))
        scores = model.predict([tiled_query, support_images])  # خروجی یک آرایه از شباهت‌ها است

        score_dict = defaultdict(list)
        for score, label in zip(scores, support_labels):
            score_dict[label].append(score)

        aggregated_scores = {label: np.mean(score_list) for label, score_list in score_dict.items()}
        
        predicted_class = max(aggregated_scores, key=aggregated_scores.get)
        predictions.append(predicted_class)

    return predictions

predictions = classify_images(support_images, support_labels, query_images, siamese_net)

for i, pred in enumerate(predictions):
    print(f" Image {i + 1}: Predicted Class -> {pred}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
 Image 1: Predicted Class -> 0
 Image 2: Predicted Class -> 0
 Image 3: Predicted Class -> 0
